In [1]:
import os
from pathlib import Path
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

In [2]:
# 1. Set device
# -----------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [3]:
# 2. Data paths
# -----------------
data_dir = Path(r'/content/drive/MyDrive/Colab Notebooks/CNN/casting_data')
train_dir = data_dir / 'train'
test_dir  = data_dir / 'test'

In [4]:
# 3. Hyperparameters
# -----------------
batch_size = 16
learning_rate = 1e-4  # lower LR for stability
epochs = 20

In [5]:
# 4. Transforms
# -----------------
common_transforms = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

data_augment = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomHorizontalFlip(),
    common_transforms
])

In [6]:
# 5. Datasets and Splits
# -----------------
full_train = ImageFolder(root=str(train_dir), transform=data_augment)
print(f"Classes: {full_train.classes}, Mapping: {full_train.class_to_idx}")
train_size = int(0.8 * len(full_train))
val_size = len(full_train) - train_size
train_dataset, val_dataset = random_split(
    full_train,
    [train_size, val_size],
    generator=torch.Generator().manual_seed(42)
)

Classes: ['def_front', 'ok_front'], Mapping: {'def_front': 0, 'ok_front': 1}


In [7]:
# 6. Dataloaders
# -----------------
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(
    ImageFolder(root=str(test_dir), transform=common_transforms),
    batch_size=batch_size, shuffle=False
)

In [8]:
# 7. Model Definition
# -----------------
class CastingCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16), nn.ReLU(), nn.MaxPool2d(2),  # 256x256
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32), nn.ReLU(), nn.MaxPool2d(2),  # 128x128
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2),  # 64x64
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 64 * 64, 256), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(256, 1)  # logits output
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x).squeeze(1)

model = CastingCNN().to(device)
print(model)

CastingCNN(
  (features): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=262144, out_features=256, bias=True)
    (2): ReLU()
    (3): Dropo

In [9]:
# 8. Loss and Optimizer
# -----------------
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
# 9. Training Loop
# -----------------
def train_epoch(loader):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for batch_idx, (images, labels) in enumerate(tqdm(loader, desc='Train', leave=False)):
        images, labels = images.to(device), labels.float().to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Debug first batch
        if batch_idx == 0:
            print("Debug -> loss:", loss.item(),
                  "min/max logits:", outputs.min().item(), outputs.max().item(),
                  "labels:", torch.unique(labels).tolist())

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        preds = (torch.sigmoid(outputs) >= 0.5).float()
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    return running_loss / total, correct / total

In [11]:
# 10. Validation Loop
# -----------------
def eval_epoch(loader, split='Val'):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(loader, desc=split, leave=False):
            images, labels = images.to(device), labels.float().to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
            preds = (torch.sigmoid(outputs) >= 0.5).float()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return running_loss / total, correct / total

In [12]:
# 11. Main Training
# -----------------
best_val_acc = 0.0
for epoch in range(1, epochs + 1):
    print(f"\nEpoch {epoch}/{epochs}")
    train_loss, train_acc = train_epoch(train_loader)
    val_loss, val_acc = eval_epoch(val_loader)
    print(f"Train Loss: {train_loss:.4f}, Acc: {train_acc*100:.2f}%")
    print(f"Val   Loss: {val_loss:.4f}, Acc: {val_acc*100:.2f}%")
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_casting_model.pth')


Epoch 1/20


Train:   0%|          | 0/332 [00:00<?, ?it/s]

Debug -> loss: 0.6759393811225891 min/max logits: -0.3319059908390045 0.2989255487918854 labels: [0.0, 1.0]


Train Loss: 0.5212, Acc: 83.43%
Val   Loss: 0.2337, Acc: 90.35%

Epoch 2/20


Train:   0%|          | 0/332 [00:00<?, ?it/s]

...

Debug -> loss: 0.066475048661232 min/max logits: -48.1768913269043 24.484737396240234 labels: [0.0, 1.0]


Train Loss: 0.0497, Acc: 98.30%
Val   Loss: 0.0113, Acc: 99.77%

Epoch 20/20


Train:   0%|          | 0/332 [00:00<?, ?it/s]

Debug -> loss: 0.06072944402694702 min/max logits: -74.26052856445312 15.920184135437012 labels: [0.0, 1.0]


Train Loss: 0.0588, Acc: 97.95%
Val   Loss: 0.0151, Acc: 99.85%


In [13]:
# 12. Test Evaluation
# -----------------
model.load_state_dict(torch.load('best_casting_model.pth'))
test_loss, test_acc = eval_epoch(test_loader, split='Test')
print(f"\nTest  Loss: {test_loss:.4f}, Acc: {test_acc*100:.2f}%")


Test  Loss: 0.0140, Acc: 99.58%
